### 5-1

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import torch.nn.functional as F
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
# load and preprocess the MNIST for anamoly detection
from torch.utils.data import DataLoader
import torch
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
mnist_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# filter the dataset for digits 1,3,5,7
target_digits = [1, 3, 5, 7]
indices = [i for i, (img, label) in enumerate(mnist_data) if label in target_digits]
filtered_dataset = Subset(mnist_data, indices)

label_mapping = {1: 0, 3: 1, 5: 2, 7: 3}

def map_labels(batch):
    inputs, labels = zip(*batch)
    labels = torch.tensor([label_mapping[label] for label in labels])
    inputs = torch.stack(inputs)
    return inputs, labels

# dataloader for the filtered dataset
train_loader = DataLoader(filtered_dataset, batch_size=64, shuffle=True, collate_fn=map_labels)

In [3]:
# train an image classifier that can recognize 1,3,5,7 then use it to do abnormal detection on the handwrittrn images other than 1,3,5,7
import torch.optim as optim
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
net = net.to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(net.parameters(), lr=0.001)


In [4]:
import torch
from tqdm import tqdm
num_epochs = 10
for epoch in range(num_epochs):
    total = 0
    correct = 0
    train_loss = 0

    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
        train_loss += loss.item()

    train_acc = correct / total

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss / len(train_loader):.4f}, Train Accuracy: {train_acc:.4f}')

print('Finished Training')


100%|██████████| 384/384 [00:03<00:00, 111.81it/s]


Epoch [1/10], Train Loss: 0.1295, Train Accuracy: 0.9605


100%|██████████| 384/384 [00:02<00:00, 145.82it/s]


Epoch [2/10], Train Loss: 0.0287, Train Accuracy: 0.9906


100%|██████████| 384/384 [00:02<00:00, 144.82it/s]


...

Epoch [8/10], Train Loss: 0.0084, Train Accuracy: 0.9976


100%|██████████| 384/384 [00:02<00:00, 145.45it/s]


Epoch [9/10], Train Loss: 0.0072, Train Accuracy: 0.9975


100%|██████████| 384/384 [00:02<00:00, 146.13it/s]

Epoch [10/10], Train Loss: 0.0066, Train Accuracy: 0.9978
Finished Training
